In [1]:
from KeywordSearch import loader, indexing, utils, kwsearch
import gc
import scipy.sparse
import numpy as np

In [4]:
print(len(kwsearch.bool_search("fine AND (okay AND good) AND happy AND fun")))

321


In [3]:
%load_ext line_profiler

%lprun -f kwsearch.bool_search kwsearch.bool_search("fine AND (okay AND good) AND happy")

Timer unit: 1e-07 s

Total time: 0.004923 s
File: c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\kwsearch.py
Function: bool_search at line 33

Line #      Hits         Time  Per Hit   % Time  Line Contents
    33                                           def bool_search(query: str) -> set:
    34                                               # print(regex_bracket.split(query))
    35         1         41.0     41.0      0.1      tokens = (bool_search_atomic(token) for token in regex_bracket.split(query) if token)
    36         1          2.0      2.0      0.0      is_not = is_and = is_or = False
    37         1          2.0      2.0      0.0      not_first = False
    38         1      17233.0  17233.0     35.0      valid, (is_not, is_and, is_or) = next(tokens)
    39         4      27058.0   6764.5     55.0      for token_eval, (is_not_, is_and_, is_or_) in tokens:
    40         3         15.0      5.0      0.0          is_not |= is_not_; is_and |= is_and_; is_or 

In [3]:
kwsearch.book_index[i]

array([], dtype=uint32)

In [10]:
x = {kwsearch.all_tokens[i] for i, token in enumerate(np.ravel(kwsearch.lookup_table[:, 208].toarray())) if i and token}

In [10]:
from KeywordSearch.kwsearch import stemmer, token_index_dict, lookup_table

In [18]:
lookup_table[1, slice(None)].indices

array([  648,   662,   669,   673,  1838,  3141,  4854,  4860,  4885,
        4900,  5242,  5245,  5678,  5680,  5772,  6344,  7488,  9823,
       10357, 10657, 10660, 10806, 11397, 12078, 12770, 14019, 14587,
       14672, 14754, 16464, 16479, 20096, 22693, 25761, 27348, 27509,
       27558, 27559, 27560, 27638, 28189, 28413, 29077, 29233, 29386,
       29765, 32030, 32767, 33239, 33439, 33504, 34081, 34122, 34853,
       35829, 35830, 36595, 37665, 38041, 38877, 39500, 39775, 40860,
       41073, 41409, 42329, 42715, 43060, 43416, 43626, 43705, 45781,
       46354, 46565, 46917, 46937, 47787, 48952, 49608, 50292, 51134,
       52873, 53793, 53935, 54616, 54966, 55025, 55819, 55989, 56631,
       58602, 61956, 62119, 64866, 69018, 70419, 71258, 71431])

In [22]:
%timeit _, valid = lookup_table[1, :].nonzero()

51.6 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
%timeit _, valid = np.nonzero(lookup_table[1, :])

51.7 µs ± 832 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
%timeit lookup_table[1, :].indices

26.3 µs ± 711 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [40]:
%timeit set(lookup_table[1, :].indices) | set(lookup_table[2, :].indices)

126 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [44]:
%timeit set(lookup_table[1, :].indices.tolist() + lookup_table[2, :].indices.tolist())

81.1 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [51]:
%timeit lookup_table[1, list(set(lookup_table[1, :].indices.tolist() + lookup_table[2, :].indices.tolist()))]

365 µs ± 7.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
%timeit lookup_table[1, tuple(set(lookup_table[1, :].indices.tolist() + lookup_table[2, :].indices.tolist()))]

366 µs ± 7.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [35]:
x = lookup_table[1, :].indices
x[x%2==1 | x%2==0]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [38]:
lookup_table[2, np.logical_or(lookup_table[1, :].indices, lookup_table[1, :].indices)]

<1x98 sparse matrix of type '<class 'numpy.bool_'>'
	with 4 stored elements in Compressed Sparse Row format>

In [57]:
stemmer.stemWord("   hello ")

'   hello '

In [6]:
import re
import numpy as np

regex_bracket = re.compile(r"\((.+)\)( +(?:NOT|AND|OR))?")
regex_bool_op = re.compile(r"(NOT|AND|OR)")

operators = frozenset(("NOT", "AND", "OR"))
np.ndarray([]).shape

()

In [7]:
from KeywordSearch.kwsearch import stemmer, token_index_dict, lookup_table
from KeywordSearch.indexing import stopwords_set

all_elems_list = list(range(lookup_table.shape[1]))
all_elems_set = frozenset(all_elems_list)
all_elems_arr = np.array(all_elems_list, dtype=np.int32)
del all_elems_list

def bool_search(query: str) -> set:
    # print(regex_bracket.split(query))
    tokens = (bool_search_atomic(token) for token in regex_bracket.split(query) if token)
    is_not = is_and = is_or = False
    not_first = False
    valid, (is_not, is_and, is_or) = next(tokens)
    for token_eval, (is_not_, is_and_, is_or_) in tokens:
        is_not |= is_not_; is_and |= is_and_; is_or |= is_or_
        if isinstance(token_eval, list):
            continue
        # print(f"Parse: {len(token_eval)} {'OR' if is_or else ''} {'AND' if is_and else ''} {'NOT' if is_not else ''}")
        if is_or:
            if is_not:
                if not_first:
                    valid = token_eval | (all_elems_set - valid) 
                else:
                    valid |= (all_elems_set - token_eval)
                is_not = False
            else:
                valid |= token_eval
            is_or = is_not = False
        elif is_and:
            if is_not:
                if not_first:
                    valid = token_eval - valid
                else:
                    valid -= token_eval
                is_not = False
            else:
                valid &= token_eval
            is_and = is_not = False
        elif is_not:
            valid -= token_eval
        else:
            print("Grammar error?")
    return valid

def bool_search_atomic(query: str) -> set:
    query = query.strip()
    # print(query)
    if not query:
        return set(), (False, False, False)
    tokens = [token for token in regex_bool_op.split(query) if token]
    is_not = is_and = is_or = False
    not_first = False
    valid = []
    for token in tokens:
        if token in operators:
            if token == "OR":
                is_or = True
                if is_not: not_first = True
            elif token == "NOT":
                is_not = True
                not_first = False
            elif token == "AND":
                is_and = True
                if is_not: not_first = True
        else:
            token_id = token_index_dict[stemmer.stemWord(token.strip().casefold())]
            if token_id:
                token_eval = lookup_table[token_id, :].indices
                if is_or:
                    if is_not:
                        if not_first:
                            valid = np.setdiff1d(all_elems_arr, valid, assume_unique=True)
                        else:
                            token_eval = np.setdiff1d(all_elems_arr, token_eval, assume_unique=True)
                        is_not = False
                    valid = np.union1d(valid, token_eval)
                    
                    is_or = False
                    if not valid.shape: return set(), (is_not, is_and, is_or)
                elif is_and:
                    if is_not:
                        if not_first:
                            valid = np.setdiff1d(token_eval, valid, assume_unique=True)
                        else:
                            valid = np.setdiff1d(valid, token_eval, assume_unique=True)
                        is_not = False
                    else:
                        valid = np.intersect1d(valid, token_eval, assume_unique=True)
                    
                    is_and = False
                    if not valid.shape: return set(), (is_not, is_and, is_or)
                else:
                    valid = token_eval
            else:
                # unseen word or stopword
                is_not = is_or = False
    if isinstance(valid, np.ndarray):
        valid = set(valid.tolist())
    return valid, (is_not, is_and, is_or)


In [4]:
len(bool_search("fine AND (okay AND good) AND happy"))

NameError: name 'bool_search' is not defined

In [145]:
a = b = c = False
a |= True
a, b, c

(True, False, False)

In [130]:
re.split(r"\((.+)\)( +(?:NOT|AND|OR))?", "fine AND (okay AND good) OR gutenberg")

['fine AND ', 'okay AND good', ' OR', ' gutenberg']

In [97]:
np.setdiff1d([1,2,3], [1], assume_unique=True)

array([2, 3])

In [86]:
lookup_table[2, :] - lookup_table[1, :]

<1x72533 sparse matrix of type '<class 'numpy.bool_'>'
	with 973 stored elements in Compressed Sparse Row format>

In [102]:
last = lookup_table[1, :].indices
lookup_table[2, last].indices.dtype

dtype('int32')

In [101]:
lookup_table[1, 0]

False

In [92]:
lookup_table[2, :] & (~lookup_table[1, :])

TypeError: unsupported operand type(s) for &: 'csr_matrix' and 'csr_matrix'

In [87]:
x = np.array([1, 1, 0, 0], dtype=np.bool_)
y = np.array([1, 0, 1, 0], dtype=np.bool_)

In [89]:
x ^ y

array([False,  True,  True, False])

In [90]:
x & (~y)

array([False,  True, False, False])

In [9]:
regex_bracket.split("((3) + 4)")

['', '(3) + 4', '']

In [4]:
regex_bool_op.split("NOT 3 OR 4 AND 5")

['', 'NOT', ' 3 ', 'OR', ' 4 ', 'AND', ' 5']

In [ ]:
lookup_table: scipy.sparse.csc_matrix = scipy.sparse.load_npz("lookup_table.npz")

In [ ]:
table_arr = scipy.sparse.csc_array(lookup_table)

In [ ]:
x = table_arr[[1], :] == True

In [ ]:
x[[0]].ndim

2

In [ ]:
x.toarray()

array([[False, False, False, ..., False, False, False]])

In [ ]:
table_arr[2, np.nonzero(x)]

<1x196 sparse array of type '<class 'numpy.bool_'>'
	with 22 stored elements in Compressed Sparse Column format>

In [ ]:
import scipy.sparse
scipy.sparse.csc_matrix([[0, 1], [2, 3]])[1, :]

<1x2 sparse matrix of type '<class 'numpy.intc'>'
	with 2 stored elements in Compressed Sparse Column format>

In [ ]:
scipy.sparse.csc_matrix([[0, 1], [2, 3]])[1, :]

<1x2 sparse matrix of type '<class 'numpy.intc'>'
	with 2 stored elements in Compressed Sparse Column format>

In [ ]:
segment_size = 5000
part_size = 5000

# for i in range(13):
#     gc.collect()
#     print(f"\nPart {i:02d}", flush=True)
#     indexing.build_full_index(offset=i*part_size, k=part_size, batch_size=segment_size, index_type="inverted", prefix=f"part{i:02d}", unsafe_pickle=False)

In [ ]:
# sizes = utils.measure_sizes()

In [ ]:
# list(sizes.items())[-1]

In [ ]:
# indexing.merge_index()

In [ ]:
gc.collect()
index = loader.load_merged_index(save_merged=False)

1140 segments to load
Finished merging 1140 segments...
Garbage collection done
All done


In [ ]:
gc.collect()
from pympler.asizeof import asizeof
index_size = asizeof(index)
mb = 1024 * 1024
gb = mb * 1024
gb_size = index_size // gb
mb_size = round((index_size % gb) / mb)
print(f"{gb_size}GB" + f" {mb_size}MB" if mb_size else '')

44GB 370MB


In [ ]:
all_tokens = loader.load_token_vocab(k=-1)

In [ ]:
import pickle
with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

In [ ]:
len(all_tokens), len(index)

(5696023, 5696023)

In [ ]:
from typing import Iterable

import numpy as np
import scipy.sparse
try:
    from tqdm.notebook import tqdm
    USE_TQDM = True
except:
    USE_TQDM = False

def construct_bool_table(index: Iterable[dict], save_path: str|None=None):
    table = scipy.sparse.dok_matrix((len(all_tokens), max(valid_books) + 1), dtype=np.bool_)
    print(len(index), table.shape)
    length = len(all_tokens)
    tqdm_iter = enumerate(index[:length])
    if USE_TQDM:
        tqdm_iter = tqdm(tqdm_iter, total=length)
    for token_id, token_dict in tqdm_iter:
        if token_dict:
            table[token_id, tuple(token_dict.keys())] = True
    gc.collect()
    table = table.tocsr()
    if save_path is None:
        return table
    else:
        gc.collect()
        scipy.sparse.save_npz("lookup_table.npz", table, compressed=True)
        return table

In [ ]:
table = construct_bool_table(index)
scipy.sparse.save_npz("lookup_table.npz", table, compressed=True)

5696023 (5696023, 72533)


  0%|          | 0/5696023 [00:00<?, ?it/s]

In [ ]:
(table.data.nbytes + table.indptr.nbytes + table.indices.nbytes) // 1024 // 1024

1063

In [ ]:
dok_table: scipy.sparse.dok_matrix = table.todok()

In [ ]:
asizeof(dok_table) // 1024 // 1024

In [ ]:
asizeof(table) // 1024 // 1024

In [ ]:
scipy.sparse.save_npz("lookup_table_coo.npz", table.tocoo(), compressed=True)

In [ ]:
x = list(range(5696023))
y = []
index_size = len(x)
batch_size = 5000
batches = index_size // batch_size
for i in range(batches):
    start = i * batch_size
    end = min(start + batch_size, index_size)
    y.extend(x[start:end])

y.extend(x[end:])
x == y, end

(True, 5695000)

In [ ]:
with open("all_tokens_old.pkl", 'rb') as f:
    _, _, all_tokens_old = pickle.load(f)
len(all_tokens_old)

5696023

In [ ]:
# inverted_index_2, indexed_books_2 = indexing.build_full_index(offset=5000, k=5000, batch_size=-1, index_type="inverted", prefix="part02")
# import h5py
# def save_inv_index_HDF5(filename: str, index: list[dict], **kwargs):
#     with h5py.File(filename, 'w') as f:
#         for i, entry in enumerate(index):
#             group = f.create_group(str(i))
#             for book_id, occurrences in entry.items():
#                 group.create_dataset(str(book_id), data=occurrences, **kwargs)
# save_inv_index_HDF5("test.h5", inverted_index_1, chunks=True, compression="gzip")

In [ ]:
import pickle
import numpy as np

with open("index/inverted_113.pkl", "rb") as f:
    tmp = pickle.load(f)

In [ ]:
len(tmp)


In [ ]:
#bow_index, indexed_books = indexing.build_full_index(k=-1)

In [ ]:
import pickle
import numpy as np

with open("index.pkl", "rb") as f:
    bow_index = pickle.load(f)
# dummy_arr = np.array([], dtype=np.uint8)
# dummy = (0, dummy_arr, 0, dummy_arr)
# bow_index = tuple([dummy] + [pair if isinstance(pair, tuple) else dummy for pair in bow_index[1:]])
# with open("index.pkl", "wb") as f:
#     pickle.dump(bow_index, f)

with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

book_list = sorted(valid_books)

non_empty_books = []
non_empty_index = []
for book_id, book_bow in enumerate(bow_index):
    if book_bow:
        non_empty_books.append(book_id)
        non_empty_index.append(book_bow)

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity
import concurrent.futures

In [ ]:
# corpus = [utils.deltazip([vocab, counts], [vocab_delta, count_delta]) for vocab_delta, vocab, count_delta, counts in non_empty_index]
# tfidf_model = TfidfModel(corpus, smartirs='ntc')
# with open("tfidf_model.pkl", "wb") as f:
#     pickle.dump(tfidf_model, f)
with open("tfidf_model.pkl", "rb") as f:
    tfidf_model = pickle.load(f)

In [ ]:
dim = len(all_tokens) + 1
def cast2numpy(x):
    arr = np.zeros(dim, dtype=np.float16)
    for i, score in x:
        arr[i] = score
    return arr

In [ ]:
tmp = [tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])] for vocab_delta, vocab, count_delta, counts in non_empty_index]
# with open("tfidf_index.pkl", "rb") as f:
#     tmp = pickle.load(f)

In [ ]:
tfidf_model[[(0,1),(3,8)]]

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[0]
cast2numpy(tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])]).mean()

In [ ]:
dim = len(all_tokens) + 1

In [ ]:
m1 = SparseMatrixSimilarity(tmp, num_best=10, num_features=dim)

In [ ]:
with open("tfidf_index.pkl", "wb") as f:
    pickle.dump(tmp, f)
with open("matrix.pkl", "wb") as f:
    pickle.dump(m1, f)

In [ ]:
asizeof.asizeof(tmp)/1024/1024/1024

In [ ]:
asizeof.asizeof(m1)/1024/1024/1024

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[269]
next(utils.deltazip([vocab, counts], [vocab_delta, count_delta]))

In [ ]:
non_empty_index[269]

In [ ]:
tokens = [1, 3, 1, 9, 3, 5654]
vocab_list = sorted(set(tokens))
token_arr = np.array(tokens)
vocab, vocab_delta = utils.cast2intarr(vocab_list)
counts, count_delta = utils.cast2intarr([np.sum(token_arr == token) for token in vocab_list])
(vocab_delta, vocab, count_delta, counts, vocab_list, [(vocab == token-vocab_delta) for token in vocab_list])
vocab + vocab_delta, counts + count_delta

In [ ]:
book_path_template = loader.token_dir + "PG%d_tokens.txt"

def read_tokens(PG_id: int):
    with open(book_path_template %PG_id, encoding="UTF-8", errors="ignore") as f:
        return loader.stemmer.stemWords(f.read().splitlines())

In [ ]:


docs = []
docs_index = []
failed_jobs = 0
complete_counter = 0
with concurrent.futures.ThreadPoolExecutor() as pool:
    jobs = {
        pool.submit(
            read_tokens, book_id)
            : book_id for book_id in indexed_books
        }
    
    for job in concurrent.futures.as_completed(jobs):
        book_id = jobs[job]
        try:
            result = job.result()
            docs.append(result)
            docs_index.append(book_id)
        except Exception as e:
            # raise e
            failed_jobs.append(book_id)
        complete_counter += 1
        print(f"Finished fetching tokens in {complete_counter} books...", end="\r")
        
        print(f"\n{len(failed_jobs)}/{len(jobs)} token fetching jobs failed")

In [ ]:
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
del docs
tfidf = TfidfModel(corpus, smartirs='ntc')
tfidf_corpus = [tfidf[doc] for doc in corpus]
del corpus, tfidf
index = MatrixSimilarity(tfidf_corpus, num_features=len(dictionary))

In [ ]:
# import pickle
# with open("all_tokens.pkl", 'rb') as f:
#     k, offset, _all_tokens = pickle.load(f)
# with open("all_tokens.pkl", "wb") as f:
#     pickle.dump((k, offset, tuple(_all_tokens)), f)

In [ ]:
from pympler import asizeof
asizeof.asizeof(bow_index), asizeof.asizeof(index)

In [ ]:
415963552/1024/1024

In [ ]:
415971552/1024/1024

In [ ]:

mem_size_lst = asizeof.asizeof(all_tokens)
mem_size_tup = asizeof.asizeof(tuple(all_tokens))
mem_size_lst/1024/1024, mem_size_tup

In [ ]:
from math import log2
import numpy as np
log2(len(all_tokens))
x = np.array([1,2,3])
x.shape[0]

In [ ]:
#loader.build_full_index()

In [ ]:
import pickle

In [ ]:
with open("index.pkl", "rb") as f:
    index = pickle.load(f)

In [ ]:
from pympler import asizeof
mem_size = asizeof.asizeof(index)

In [ ]:
mem_size/1024/1024

In [ ]:
index[1]

In [ ]:
size = 0
print(len(index))
for i, token_eval in enumerate(index):
    for arr in token_eval.values():
        size += asizeof.asizeof(arr)
    print(i, end='\r')
size

In [ ]:
size/1024/1024